# NeighborSampler

In [ ]:
"""Data loading components for neighbor sampling"""
from ..base import NID, EID
from ..transforms import to_block
from .base import BlockSampler

import time

class NeighborSampler(BlockSampler):
    """Sampler that builds computational dependency of node representations via
    neighbor sampling for multilayer GNN.

    This sampler will make every node gather messages from a fixed number of neighbors
    per edge type.  The neighbors are picked uniformly.

    Parameters
    ----------
    fanouts : list[int] or list[dict[etype, int]]
        List of neighbors to sample per edge type for each GNN layer, with the i-th
        element being the fanout for the i-th GNN layer.

        If only a single integer is provided, DGL assumes that every edge type
        will have the same fanout.

        If -1 is provided for one edge type on one layer, then all inbound edges
        of that edge type will be included.
    edge_dir : str, default ``'in'``
        Can be either ``'in' `` where the neighbors will be sampled according to
        incoming edges, or ``'out'`` otherwise, same as :func:`dgl.sampling.sample_neighbors`.
    prob : str, optional
        If given, the probability of each neighbor being sampled is proportional
        to the edge feature value with the given name in ``g.edata``.  The feature must be
        a scalar on each edge.

        This argument is mutually exclusive with :attr:`mask`.  If you want to
        specify both a mask and a probability, consider multiplying the probability
        with the mask instead.
    mask : str, optional
        If given, a neighbor could be picked only if the edge mask with the given
        name in ``g.edata`` is True.  The data must be boolean on each edge.

        This argument is mutually exclusive with :attr:`prob`.  If you want to
        specify both a mask and a probability, consider multiplying the probability
        with the mask instead.
    replace : bool, default False
        Whether to sample with replacement
    prefetch_node_feats : list[str] or dict[ntype, list[str]], optional
        The source node data to prefetch for the first MFG, corresponding to the
        input node features necessary for the first GNN layer.
    prefetch_labels : list[str] or dict[ntype, list[str]], optional
        The destination node data to prefetch for the last MFG, corresponding to
        the node labels of the minibatch.
    prefetch_edge_feats : list[str] or dict[etype, list[str]], optional
        The edge data names to prefetch for all the MFGs, corresponding to the
        edge features necessary for all GNN layers.
    output_device : device, optional
        The device of the output subgraphs or MFGs.  Default is the same as the
        minibatch of seed nodes.

    Examples
    --------
    **Node classification**

    To train a 3-layer GNN for node classification on a set of nodes ``train_nid`` on
    a homogeneous graph where each node takes messages from 5, 10, 15 neighbors for
    the first, second, and third layer respectively (assuming the backend is PyTorch):

    >>> sampler = dgl.dataloading.NeighborSampler([5, 10, 15])
    >>> dataloader = dgl.dataloading.DataLoader(
    ...     g, train_nid, sampler,
    ...     batch_size=1024, shuffle=True, drop_last=False, num_workers=4)
    >>> for input_nodes, output_nodes, blocks in dataloader:
    ...     train_on(blocks)

    If training on a heterogeneous graph and you want different number of neighbors for each
    edge type, one should instead provide a list of dicts.  Each dict would specify the
    number of neighbors to pick per edge type.

    >>> sampler = dgl.dataloading.NeighborSampler([
    ...     {('user', 'follows', 'user'): 5,
    ...      ('user', 'plays', 'game'): 4,
    ...      ('game', 'played-by', 'user'): 3}] * 3)

    If you would like non-uniform neighbor sampling:

    >>> g.edata['p'] = torch.rand(g.num_edges())   # any non-negative 1D vector works
    >>> sampler = dgl.dataloading.NeighborSampler([5, 10, 15], prob='p')

    Or sampling on edge masks:

    >>> g.edata['mask'] = torch.rand(g.num_edges()) < 0.2   # any 1D boolean mask works
    >>> sampler = dgl.dataloading.NeighborSampler([5, 10, 15], prob='mask')

    **Edge classification and link prediction**

    This class can also work for edge classification and link prediction together
    with :func:`as_edge_prediction_sampler`.

    >>> sampler = dgl.dataloading.NeighborSampler([5, 10, 15])
    >>> sampler = dgl.dataloading.as_edge_prediction_sampler(sampler)
    >>> dataloader = dgl.dataloading.DataLoader(
    ...     g, train_eid, sampler,
    ...     batch_size=1024, shuffle=True, drop_last=False, num_workers=4)

    See the documentation :func:`as_edge_prediction_sampler` for more details.

    Notes
    -----
    For the concept of MFGs, please refer to
    :ref:`User Guide Section 6 <guide-minibatch>` and
    :doc:`Minibatch Training Tutorials <tutorials/large/L0_neighbor_sampling_overview>`.
    """
    def __init__(self, fanouts, edge_dir='in', prob=None, mask=None, replace=False,
                 prefetch_node_feats=None, prefetch_labels=None, prefetch_edge_feats=None,
                 output_device=None):
        super().__init__(prefetch_node_feats=prefetch_node_feats,
                         prefetch_labels=prefetch_labels,
                         prefetch_edge_feats=prefetch_edge_feats,
                         output_device=output_device)
        self.fanouts = fanouts
        self.edge_dir = edge_dir
        if mask is not None and prob is not None:
            raise ValueError(
                    'Mask and probability arguments are mutually exclusive. '
                    'Consider multiplying the probability with the mask '
                    'to achieve the same goal.')
        self.prob = prob or mask
        self.replace = replace

    def sample_blocks(self, g, seed_nodes, exclude_eids=None):

        start = time.time()
        sample_time = 0

        output_nodes = seed_nodes
        blocks = []
        for fanout in reversed(self.fanouts):
            frontier = g.sample_neighbors(
                seed_nodes, fanout, edge_dir=self.edge_dir, prob=self.prob,
                replace=self.replace, output_device=self.output_device,
                exclude_edges=exclude_eids)
            eid = frontier.edata[EID]
            block = to_block(frontier, seed_nodes)
            block.edata[EID] = eid
            seed_nodes = block.srcdata[NID]
            blocks.insert(0, block)

        end = time.time()
        sample_time = end-start
        # print("{:.4f}".format(sample_time))

        return seed_nodes, output_nodes, blocks

class NeighborDFSampler(BlockSampler):

    def __init__(self, fanouts, edge_dir='in', prob=None, mask=None, replace=False,
                 prefetch_node_feats=None, prefetch_labels=None, prefetch_edge_feats=None,
                 output_device=None):
        super().__init__(prefetch_node_feats=prefetch_node_feats,
                         prefetch_labels=prefetch_labels,
                         prefetch_edge_feats=prefetch_edge_feats,
                         output_device=output_device)
        self.fanouts = fanouts
        self.edge_dir = edge_dir
        if mask is not None and prob is not None:
            raise ValueError(
                    'Mask and probability arguments are mutually exclusive. '
                    'Consider multiplying the probability with the mask '
                    'to achieve the same goal.')
        self.prob = prob or mask
        self.replace = replace

    def sample_blocks(self, g, seed_nodes, exclude_eids=None):

        start = time.time()
        sample_time = 0

        output_nodes = seed_nodes
        blocks = []
        # g must be DisGraph instance

        frontiers = g.df_sample_neighbors(
            seed_nodes, self.fanouts, edge_dir=self.edge_dir, prob=self.prob,
            replace=self.replace, output_device=self.output_device,
            exclude_edges=exclude_eids)

        for frontier in frontiers:
            eid = frontier.edata[EID]
            block = to_block(frontier, seed_nodes)
            block.edata[EID] = eid
            seed_nodes = block.srcdata[NID]
            blocks.insert(0, block)

        end = time.time()
        sample_time = end-start
        # print("{:.4f}".format(sample_time))

        return seed_nodes, output_nodes, blocks

MultiLayerNeighborSampler = NeighborSampler

class MultiLayerFullNeighborSampler(NeighborSampler):
    """Sampler that builds computational dependency of node representations by taking messages
    from all neighbors for multilayer GNN.

    This sampler will make every node gather messages from every single neighbor per edge type.

    Parameters
    ----------
    n_layers : int
        The number of GNN layers to sample.
    kwargs :
        Passed to :class:`dgl.dataloading.NeighborSampler`.

    Examples
    --------
    To train a 3-layer GNN for node classification on a set of nodes ``train_nid`` on
    a homogeneous graph where each node takes messages from all neighbors for the first,
    second, and third layer respectively (assuming the backend is PyTorch):

    >>> sampler = dgl.dataloading.MultiLayerFullNeighborSampler(3)
    >>> dataloader = dgl.dataloading.DataLoader(
    ...     g, train_nid, sampler,
    ...     batch_size=1024, shuffle=True, drop_last=False, num_workers=4)
    >>> for input_nodes, output_nodes, blocks in dataloader:
    ...     train_on(blocks)

    Notes
    -----
    For the concept of MFGs, please refer to
    :ref:`User Guide Section 6 <guide-minibatch>` and
    :doc:`Minibatch Training Tutorials <tutorials/large/L0_neighbor_sampling_overview>`.
    """
    def __init__(self, num_layers, **kwargs):
        super().__init__([-1] * num_layers, **kwargs)


# graph_services

In [ ]:
"""A set of graph services of getting subgraphs from DistGraph"""
from collections import namedtuple

import numpy as np
import time

from .. import backend as F
from ..base import EID, NID
from ..convert import graph, heterograph
from ..sampling import sample_etype_neighbors as local_sample_etype_neighbors
from ..sampling import sample_neighbors as local_sample_neighbors
from ..subgraph import in_subgraph as local_in_subgraph
from ..utils import toindex
from .. import backend as F
from .rpc import (
    Request,
    Response,
    recv_responses,
    register_service,
    send_requests_to_machine,
)

__all__ = [
    "sample_neighbors",
    "sample_etype_neighbors",
    "in_subgraph",
    "find_edges",
]

SAMPLING_SERVICE_ID = 6657
INSUBGRAPH_SERVICE_ID = 6658
EDGES_SERVICE_ID = 6659
OUTDEGREE_SERVICE_ID = 6660
INDEGREE_SERVICE_ID = 6661
ETYPE_SAMPLING_SERVICE_ID = 6662
DFSAMPLING_SERVICE_ID = 6666

class DFSubgraphResponse(Response):
    """The response for sampling and in_subgraph"""

    def __init__(self, LocalSampledGraphs, bucket):
        self.LocalSampledGraphs = LocalSampledGraphs
        self.bucket = bucket

    def __setstate__(self, state):
        self.LocalSampledGraphs, self.bucket = state

    def __getstate__(self):
        return self.LocalSampledGraphs, self.bucket

class SubgraphResponse(Response):
    """The response for sampling and in_subgraph"""

    def __init__(self, global_src, global_dst, global_eids):
        self.global_src = global_src
        self.global_dst = global_dst
        self.global_eids = global_eids

    def __setstate__(self, state):
        self.global_src, self.global_dst, self.global_eids = state

    def __getstate__(self):
        return self.global_src, self.global_dst, self.global_eids

class FindEdgeResponse(Response):
    """The response for sampling and in_subgraph"""

    def __init__(self, global_src, global_dst, order_id):
        self.global_src = global_src
        self.global_dst = global_dst
        self.order_id = order_id

    def __setstate__(self, state):
        self.global_src, self.global_dst, self.order_id = state

    def __getstate__(self):
        return self.global_src, self.global_dst, self.order_id


def _sample_neighbors(
    local_g, partition_book, seed_nodes, fan_out, edge_dir, prob, replace
):
    """Sample from local partition.

    The input nodes use global IDs. We need to map the global node IDs to local node IDs,
    perform sampling and map the sampled results to the global IDs space again.
    The sampled results are stored in three vectors that store source nodes, destination nodes
    and edge IDs.
    """
    local_ids = partition_book.nid2localnid(seed_nodes, partition_book.partid)
    local_ids = F.astype(local_ids, local_g.idtype)
    # local_ids = self.seed_nodes
    sampled_graph = local_sample_neighbors(
        local_g,
        local_ids,
        fan_out,
        edge_dir,
        prob,
        replace,
        _dist_training=True,
    )
    global_nid_mapping = local_g.ndata[NID]
    src, dst = sampled_graph.edges()
    global_src, global_dst = F.gather_row(
        global_nid_mapping, src
    ), F.gather_row(global_nid_mapping, dst)
    global_eids = F.gather_row(local_g.edata[EID], sampled_graph.edata[EID])

    return global_src, global_dst, global_eids

def _df_sample_neighbors(
    local_g, partition_book, seed_nodes, fan_outs, edge_dir, prob, replace, bucket, layer_idx
):
    # return items
    LocalSampledGraphs = []
    local_bucket = []
    fanouts_list = []

    #fill local_bucket with empty list
    for i in range(len(fan_outs)-1):
      local_bucket.append([])

    # control fanouts: bucket에 값이 존재하는 layer부터 sampling 수행
    if isinstance(fan_outs,int):
        fanouts_list.append(fan_outs)
    else:
        fanouts_list = fan_outs

    sliced_fanouts = []
    for i in range(len(fanouts_list)-layer_idx):
        sliced_fanouts.append(fanouts_list[i])
    fan_outs = sliced_fanouts

    start = time.time()
    for i, fan_out in enumerate(reversed(fan_outs)):

        # seed_nodes를 local id로 매핑
        local_ids = partition_book.nid2localnid(seed_nodes, partition_book.partid)
        local_ids = F.astype(local_ids, local_g.idtype)

        bucket_ids = None
        # bucket이 비어있지 않으면 값들을 seed_node에 추가
        if layer_idx > 0:
            if len(bucket[layer_idx+i-1])!=0:
                # bucket의 노드 중 현재 파티션에 속한 노드만 ids로 저장

                bucket_ids = _check_local(partition_book, bucket[layer_idx+i-1])


        if bucket_ids is not None and i!=0:
            localized_bucket_ids = partition_book.nid2localnid(bucket_ids, partition_book.partid)
            localized_bucket_ids = F.astype(localized_bucket_ids, local_g.idtype)
            all_seed_nodes = []
            all_seed_nodes.extend(local_ids.tolist())
            all_seed_nodes.extend(localized_bucket_ids.tolist())
            all_seed_nodes = toindex(all_seed_nodes).tousertensor()
        else:
            all_seed_nodes = local_ids

        # local sampling
        sampled_graph = local_sample_neighbors(
            local_g,
            all_seed_nodes,
            fan_out,
            edge_dir,
            prob,
            replace,
            _dist_training=True,
        )


        # local id를 다시 global로 매핑
        global_nid_mapping = local_g.ndata[NID]
        src, dst = sampled_graph.edges()
        global_src, global_dst = F.gather_row(
            global_nid_mapping, src
        ), F.gather_row(global_nid_mapping, dst)
        global_eids = F.gather_row(local_g.edata[EID], sampled_graph.edata[EID])
        # 샘플링 결과 저장
        LocalSampledGraphs.append(LocalSampledGraph(global_src, global_dst, global_eids))

        # remote or local을 체크하여 local은 다시 seed_node에 remote는 bucket으로 삽입
        # local_nids는 local id이며 remote_list는 global id이다
        local_nids, remote_list = _check_location_local(local_g, partition_book, global_src)
        seed_nodes = local_nids
        if (layer_idx+i)<(len(fan_outs)-1) and len(remote_list)!=0:
          local_bucket[layer_idx+i].extend(remote_list)

    end = time.time()
    f = open("/home/van/disney_workplace/dgl/examples/pytorch/graphsage/dist/dfs_{}_loc_rem.txt".format("1000"),'a+')
    data = str(end-start)
    f.write(data+"\n")
    f.close()

    return LocalSampledGraphs, local_bucket

def _sample_etype_neighbors(
    local_g,
    partition_book,
    seed_nodes,
    etype_offset,
    fan_out,
    edge_dir,
    prob,
    replace,
    etype_sorted=False,
):
    """Sample from local partition.

    The input nodes use global IDs. We need to map the global node IDs to local node IDs,
    perform sampling and map the sampled results to the global IDs space again.
    The sampled results are stored in three vectors that store source nodes, destination nodes
    and edge IDs.
    """
    local_ids = partition_book.nid2localnid(seed_nodes, partition_book.partid)
    local_ids = F.astype(local_ids, local_g.idtype)

    sampled_graph = local_sample_etype_neighbors(
        local_g,
        local_ids,
        etype_offset,
        fan_out,
        edge_dir,
        prob,
        replace,
        etype_sorted=etype_sorted,
        _dist_training=True,
    )
    global_nid_mapping = local_g.ndata[NID]
    src, dst = sampled_graph.edges()
    global_src, global_dst = F.gather_row(
        global_nid_mapping, src
    ), F.gather_row(global_nid_mapping, dst)
    global_eids = F.gather_row(local_g.edata[EID], sampled_graph.edata[EID])
    return global_src, global_dst, global_eids


def _find_edges(local_g, partition_book, seed_edges):
    """Given an edge ID array, return the source
    and destination node ID array ``s`` and ``d`` in the local partition.
    """
    local_eids = partition_book.eid2localeid(seed_edges, partition_book.partid)
    local_eids = F.astype(local_eids, local_g.idtype)
    local_src, local_dst = local_g.find_edges(local_eids)
    global_nid_mapping = local_g.ndata[NID]
    global_src = global_nid_mapping[local_src]
    global_dst = global_nid_mapping[local_dst]
    return global_src, global_dst


def _in_degrees(local_g, partition_book, n):
    """Get in-degree of the nodes in the local partition."""
    local_nids = partition_book.nid2localnid(n, partition_book.partid)
    local_nids = F.astype(local_nids, local_g.idtype)
    return local_g.in_degrees(local_nids)


def _out_degrees(local_g, partition_book, n):
    """Get out-degree of the nodes in the local partition."""
    local_nids = partition_book.nid2localnid(n, partition_book.partid)
    local_nids = F.astype(local_nids, local_g.idtype)
    return local_g.out_degrees(local_nids)


def _in_subgraph(local_g, partition_book, seed_nodes):
    """Get in subgraph from local partition.

    The input nodes use global IDs. We need to map the global node IDs to local node IDs,
    get in-subgraph and map the sampled results to the global IDs space again.
    The results are stored in three vectors that store source nodes, destination nodes
    and edge IDs.
    """
    local_ids = partition_book.nid2localnid(seed_nodes, partition_book.partid)
    local_ids = F.astype(local_ids, local_g.idtype)
    # local_ids = self.seed_nodes
    sampled_graph = local_in_subgraph(local_g, local_ids)
    global_nid_mapping = local_g.ndata[NID]
    src, dst = sampled_graph.edges()
    global_src, global_dst = global_nid_mapping[src], global_nid_mapping[dst]
    global_eids = F.gather_row(local_g.edata[EID], sampled_graph.edata[EID])
    return global_src, global_dst, global_eids


# --- NOTE 1 ---
# (BarclayII)
# If the sampling algorithm needs node and edge data, ideally the
# algorithm should query the underlying feature storage to get what it
# just needs to complete the job.  For instance, with
# sample_etype_neighbors, we only need the probability of the seed nodes'
# neighbors.
#
# However, right now we are reusing the existing subgraph sampling
# interfaces of DGLGraph (i.e. single machine solution), which needs
# the data of *all* the nodes/edges.  Going distributed, we now need
# the node/edge data of the *entire* local graph partition.
#
# If the sampling algorithm only use edge data, the current design works
# because the local graph partition contains all the in-edges of the
# assigned nodes as well as the data.  This is the case for
# sample_etype_neighbors.
#
# However, if the sampling algorithm requires data of the neighbor nodes
# (e.g. sample_neighbors_biased which performs biased sampling based on the
# type of the neighbor nodes), the current design will fail because the
# neighbor nodes (hence the data) may not belong to the current partition.
# This is a limitation of the current DistDGL design.  We should improve it
# later.

class SamplingRequest(Request):
    """Sampling Request"""

    def __init__(self, nodes, fan_out, edge_dir="in", prob=None, replace=False):
        self.seed_nodes = nodes
        self.edge_dir = edge_dir
        self.prob = prob
        self.replace = replace
        self.fan_out = fan_out

    def __setstate__(self, state):
        (
            self.seed_nodes,
            self.edge_dir,
            self.prob,
            self.replace,
            self.fan_out,
        ) = state

    def __getstate__(self):
        return (
            self.seed_nodes,
            self.edge_dir,
            self.prob,
            self.replace,
            self.fan_out,
        )

    def process_request(self, server_state):
        local_g = server_state.graph
        partition_book = server_state.partition_book
        kv_store = server_state.kv_store
        if self.prob is not None:
            prob = [kv_store.data_store[self.prob]]
        else:
            prob = None
        global_src, global_dst, global_eids = _sample_neighbors(
            local_g,
            partition_book,
            self.seed_nodes,
            self.fan_out,
            self.edge_dir,
            prob,
            self.replace,
        )
        return SubgraphResponse(global_src, global_dst, global_eids)

class DFSamplingRequest(Request):
    """Sampling Request"""

    def __init__(self, nodes, fan_outs, edge_dir="in", prob=None, replace=False, bucket=None, layer_idx = None):
        self.seed_nodes = nodes
        self.edge_dir = edge_dir
        self.prob = prob
        self.replace = replace
        self.fan_outs = fan_outs
        self.bucket = bucket
        self.layer_idx = layer_idx

    def __setstate__(self, state):
        (
            self.seed_nodes,
            self.edge_dir,
            self.prob,
            self.replace,
            self.fan_outs,
            self.bucket,
            self.layer_idx
        ) = state

    def __getstate__(self):
        return (
            self.seed_nodes,
            self.edge_dir,
            self.prob,
            self.replace,
            self.fan_outs,
            self.bucket,
            self.layer_idx
        )


    def process_request(self, server_state):
        local_g = server_state.graph
        partition_book = server_state.partition_book
        kv_store = server_state.kv_store
        if self.prob is not None:
            prob = [kv_store.data_store[self.prob]]
        else:
            prob = None
        LocalSampledGraphs, local_bucket = _df_sample_neighbors(
            local_g,
            partition_book,
            self.seed_nodes,
            self.fan_outs,
            self.edge_dir,
            prob,
            self.replace,
            self.bucket,
            self.layer_idx
        )
        return DFSubgraphResponse(LocalSampledGraphs, local_bucket)

class SamplingRequestEtype(Request):
    """Sampling Request"""

    def __init__(
        self,
        nodes,
        fan_out,
        edge_dir="in",
        prob=None,
        replace=False,
        etype_sorted=True,
    ):
        self.seed_nodes = nodes
        self.edge_dir = edge_dir
        self.prob = prob
        self.replace = replace
        self.fan_out = fan_out
        self.etype_sorted = etype_sorted

    def __setstate__(self, state):
        (
            self.seed_nodes,
            self.edge_dir,
            self.prob,
            self.replace,
            self.fan_out,
            self.etype_sorted,
        ) = state

    def __getstate__(self):
        return (
            self.seed_nodes,
            self.edge_dir,
            self.prob,
            self.replace,
            self.fan_out,
            self.etype_sorted,
        )

    def process_request(self, server_state):
        local_g = server_state.graph
        partition_book = server_state.partition_book
        kv_store = server_state.kv_store
        etype_offset = partition_book.local_etype_offset
        # See NOTE 1
        if self.prob is not None:
            probs = [
                kv_store.data_store[key] if key != "" else None
                for key in self.prob
            ]
        else:
            probs = None
        global_src, global_dst, global_eids = _sample_etype_neighbors(
            local_g,
            partition_book,
            self.seed_nodes,
            etype_offset,
            self.fan_out,
            self.edge_dir,
            probs,
            self.replace,
            self.etype_sorted,
        )
        return SubgraphResponse(global_src, global_dst, global_eids)


class EdgesRequest(Request):
    """Edges Request"""

    def __init__(self, edge_ids, order_id):
        self.edge_ids = edge_ids
        self.order_id = order_id

    def __setstate__(self, state):
        self.edge_ids, self.order_id = state

    def __getstate__(self):
        return self.edge_ids, self.order_id

    def process_request(self, server_state):
        local_g = server_state.graph
        partition_book = server_state.partition_book
        global_src, global_dst = _find_edges(
            local_g, partition_book, self.edge_ids
        )

        return FindEdgeResponse(global_src, global_dst, self.order_id)


class InDegreeRequest(Request):
    """In-degree Request"""

    def __init__(self, n, order_id):
        self.n = n
        self.order_id = order_id

    def __setstate__(self, state):
        self.n, self.order_id = state

    def __getstate__(self):
        return self.n, self.order_id

    def process_request(self, server_state):
        local_g = server_state.graph
        partition_book = server_state.partition_book
        deg = _in_degrees(local_g, partition_book, self.n)

        return InDegreeResponse(deg, self.order_id)


class InDegreeResponse(Response):
    """The response for in-degree"""

    def __init__(self, deg, order_id):
        self.val = deg
        self.order_id = order_id

    def __setstate__(self, state):
        self.val, self.order_id = state

    def __getstate__(self):
        return self.val, self.order_id


class OutDegreeRequest(Request):
    """Out-degree Request"""

    def __init__(self, n, order_id):
        self.n = n
        self.order_id = order_id

    def __setstate__(self, state):
        self.n, self.order_id = state

    def __getstate__(self):
        return self.n, self.order_id

    def process_request(self, server_state):
        local_g = server_state.graph
        partition_book = server_state.partition_book
        deg = _out_degrees(local_g, partition_book, self.n)

        return OutDegreeResponse(deg, self.order_id)


class OutDegreeResponse(Response):
    """The response for out-degree"""

    def __init__(self, deg, order_id):
        self.val = deg
        self.order_id = order_id

    def __setstate__(self, state):
        self.val, self.order_id = state

    def __getstate__(self):
        return self.val, self.order_id


class InSubgraphRequest(Request):
    """InSubgraph Request"""

    def __init__(self, nodes):
        self.seed_nodes = nodes

    def __setstate__(self, state):
        self.seed_nodes = state

    def __getstate__(self):
        return self.seed_nodes

    def process_request(self, server_state):
        local_g = server_state.graph
        partition_book = server_state.partition_book
        global_src, global_dst, global_eids = _in_subgraph(
            local_g, partition_book, self.seed_nodes
        )
        return SubgraphResponse(global_src, global_dst, global_eids)


def merge_graphs(res_list, num_nodes):
    """Merge request from multiple servers"""
    if len(res_list) > 1:
        srcs = []
        dsts = []
        eids = []
        for res in res_list:
            srcs.append(res.global_src)
            dsts.append(res.global_dst)
            eids.append(res.global_eids)
        src_tensor = F.cat(srcs, 0)
        dst_tensor = F.cat(dsts, 0)
        eid_tensor = F.cat(eids, 0)
    else:
        src_tensor = res_list[0].global_src
        dst_tensor = res_list[0].global_dst
        eid_tensor = res_list[0].global_eids
    g = graph((src_tensor, dst_tensor), num_nodes=num_nodes)
    g.edata[EID] = eid_tensor
    return g


LocalSampledGraph = namedtuple(
    "LocalSampledGraph", "global_src global_dst global_eids"
)


def _distributed_access(g, nodes, issue_remote_req, local_access):
    """A routine that fetches local neighborhood of nodes from the distributed graph.

    The local neighborhood of some nodes are stored in the local machine and the other
    nodes have their neighborhood on remote machines. This code will issue remote
    access requests first before fetching data from the local machine. In the end,
    we combine the data from the local machine and remote machines.
    In this way, we can hide the latency of accessing data on remote machines.

    Parameters
    ----------
    g : DistGraph
        The distributed graph
    nodes : tensor
        The nodes whose neighborhood are to be fetched.
    issue_remote_req : callable
        The function that issues requests to access remote data.
    local_access : callable
        The function that reads data on the local machine.

    Returns
    -------
    DGLGraph
        The subgraph that contains the neighborhoods of all input nodes.
    """
    req_list = []
    partition_book = g.get_partition_book()
    nodes = toindex(nodes).tousertensor()
    partition_id = partition_book.nid2partid(nodes)
    local_nids = None
    data1 = "0"
    data2 = "0"
    for pid in range(partition_book.num_partitions()):
        node_id = F.boolean_mask(nodes, partition_id == pid)
        # We optimize the sampling on a local partition if the server and the client
        # run on the same machine. With a good partitioning, most of the seed nodes
        # should reside in the local partition. If the server and the client
        # are not co-located, the client doesn't have a local partition.
        if pid == partition_book.partid and g.local_partition is not None:
            assert local_nids is None
            local_nids = node_id
            data1 = str(len(local_nids))
        elif len(node_id) != 0:
            req = issue_remote_req(node_id)
            req_list.append((pid, req))
            data2 = str(len(node_id))


    f = open("/home/van/disney_workplace/dgl/examples/pytorch/graphsage/dist/bfs_{}_loc_rem.txt".format("1000"),'a+')
    data = str(len(nodes))
    f.write(data+"\n")
    f.write(data1+"\n")
    f.write(data2+"\n")
    f.close()


    # send requests to the remote machine.

    msgseq2pos = None

    start = time.time()

    if len(req_list) > 0:
        msgseq2pos = send_requests_to_machine(req_list)

    # sample neighbors for the nodes in the local partition.
    local_start_time = time.time()
    res_list = []
    if local_nids is not None:
        src, dst, eids = local_access(
            g.local_partition, partition_book, local_nids
        )
        res_list.append(LocalSampledGraph(src, dst, eids))

    local_end_time = time.time()
    local_sample_time = local_end_time - local_start_time

    # receive responses from remote machines.
    if msgseq2pos is not None:
        results = recv_responses(msgseq2pos)
        res_list.extend(results)

    end = time.time()
    sample_time = end-start
    f = open("/home/van/disney_workplace/dgl/examples/pytorch/graphsage/dist/bfs_{}_loc_rem.txt".format("1000"),'a+')
    data_local = str(local_sample_time)
    data = str(sample_time)
    f.write(data_local+"\n"+data+"\n")
    f.close()

    sampled_graph = merge_graphs(res_list, g.number_of_nodes())
    return sampled_graph

def _dfs_distributed_access(g, nodes, issue_remote_req, local_access, fan_outs):

    partition_book = g.get_partition_book()
    local_nids = None
    sampledGraphs = []
    for i in range(len(fan_outs)):
      sampledGraphs.append([])
    layer_idx = 0
    exist_flag = True
    num_while_loop = 1
    bucket = []
    for i in range(len(fan_outs)-1):
        bucket.append([])

    while(num_while_loop<=len(fan_outs)):
      exist_flag = False
      RemoteSampledGraphs = []
      LocalSampledGraphs = []
      req_list = []

      # check all partition ids of nodes
      local_nids, remote_list = _check_location(g, nodes)

      # send requests to the remote machine.
      msgseq2pos = None

    #   start = time.time()

      if len(remote_list) > 0:
          for remote in remote_list:
            pid = remote[0]
            node_ids_each_machine_tensor = remote[1]
            req = issue_remote_req(node_ids_each_machine_tensor, bucket, layer_idx)
            req_list.append((pid, req))
          msgseq2pos = send_requests_to_machine(req_list)

      # sample neighbors for the nodes in the local partition.
      local_start_time = time.time()
      local_bucket = []
      if local_nids is not None and len(local_nids)!=0:
          LocalSampledGraphs, local_bucket = local_access(
              g.local_partition, partition_book, local_nids, bucket, layer_idx
          )

      local_end_time = time.time()
      local_sample_time = local_end_time - local_start_time


      bucket = []
      for i in range(len(fan_outs)-1):
          bucket.append([])

      # receive responses from remote machines.

      remote_buckets = []
      if msgseq2pos is not None:
          res = recv_responses(msgseq2pos)
          for items in reversed(res):
              RemoteSampledGraphs = items.__getstate__()[0]
              remote_bucket = items.__getstate__()[1]
              for i in reversed(range(len(RemoteSampledGraphs))):
                sampledGraphs[layer_idx+i].append(RemoteSampledGraphs[i])

              for i in range(len(bucket)):
                # print("part0;_dfs_distributed_access:len(remote_bucket[i])",i,":",len(remote_bucket[i]))
                bucket[i].extend(remote_bucket[i])

    #   end = time.time()
    #   sample_time = end-start
    #   f = open("/home/van/disney_workplace/dgl/examples/pytorch/graphsage/dist/dfs_{}_loc_rem.txt".format("1000"),'a+')
    #   data_local = str(local_sample_time)
    #   data = str(sample_time)
    #   f.write(data_local+"\n"+data+"\n")
    #   f.close()

      # concatenate results


      for i in reversed(range(len(LocalSampledGraphs))):
          sampledGraphs[layer_idx+i].append(LocalSampledGraphs[i])

      if len(local_bucket)!=0:
        for i in range(len(bucket)):
            bucket[i].extend(local_bucket[i])

      # check bucket is empty or not
      for i, layer in enumerate(bucket):
        if len(layer)!=0:
          exist_flag =True

          # change nodes of first bucket
          nodes = layer
        #   print("part0;_dfs_distributed_access:len(nodes)",i,":",len(nodes))
          layer_idx = i+1
          break
      # bucket is empty so break while loop
      if exist_flag==False:

        break
      num_while_loop = num_while_loop+1

    # f = open("/home/van/disney_workplace/dgl/examples/pytorch/graphsage/dist/dfs_num_requests.txt",'a+')
    # data = str(num_while_loop)
    # f.write(data+"\n")
    # f.close()

    # merge graphs
    sampled_graphs = []
    for i in range(len(sampledGraphs)):
        sampled_graph = merge_graphs(sampledGraphs[i], g.number_of_nodes())
        sampled_graphs.append(sampled_graph)

    return sampled_graphs

def _frontier_to_heterogeneous_graph(g, frontier, gpb):
    # We need to handle empty frontiers correctly.
    if frontier.number_of_edges() == 0:
        data_dict = {
            etype: (np.zeros(0), np.zeros(0)) for etype in g.canonical_etypes
        }
        return heterograph(
            data_dict,
            {ntype: g.number_of_nodes(ntype) for ntype in g.ntypes},
            idtype=g.idtype,
        )

    etype_ids, frontier.edata[EID] = gpb.map_to_per_etype(frontier.edata[EID])
    src, dst = frontier.edges()
    etype_ids, idx = F.sort_1d(etype_ids)
    src, dst = F.gather_row(src, idx), F.gather_row(dst, idx)
    eid = F.gather_row(frontier.edata[EID], idx)
    _, src = gpb.map_to_per_ntype(src)
    _, dst = gpb.map_to_per_ntype(dst)

    data_dict = dict()
    edge_ids = {}
    for etid, etype in enumerate(g.canonical_etypes):
        type_idx = etype_ids == etid
        if F.sum(type_idx, 0) > 0:
            data_dict[etype] = (
                F.boolean_mask(src, type_idx),
                F.boolean_mask(dst, type_idx),
            )
            edge_ids[etype] = F.boolean_mask(eid, type_idx)
    hg = heterograph(
        data_dict,
        {ntype: g.number_of_nodes(ntype) for ntype in g.ntypes},
        idtype=g.idtype,
    )

    for etype in edge_ids:
        hg.edges[etype].data[EID] = edge_ids[etype]
    return hg


def sample_etype_neighbors(
    g,
    nodes,
    fanout,
    edge_dir="in",
    prob=None,
    replace=False,
    etype_sorted=True,
):
    """Sample from the neighbors of the given nodes from a distributed graph.

    For each node, a number of inbound (or outbound when ``edge_dir == 'out'``) edges
    will be randomly chosen.  The returned graph will contain all the nodes in the
    original graph, but only the sampled edges.

    Node/edge features are not preserved. The original IDs of
    the sampled edges are stored as the `dgl.EID` feature in the returned graph.

    This function assumes the input is a homogeneous ``DGLGraph`` with the edges
    ordered by their edge types. The sampled subgraph is also
    stored in the homogeneous graph format. That is, all nodes and edges are assigned
    with unique IDs (in contrast, we typically use a type name and a node/edge ID to
    identify a node or an edge in ``DGLGraph``). We refer to this type of IDs
    as *homogeneous ID*.
    Users can use :func:`dgl.distributed.GraphPartitionBook.map_to_per_ntype`
    and :func:`dgl.distributed.GraphPartitionBook.map_to_per_etype`
    to identify their node/edge types and node/edge IDs of that type.

    Parameters
    ----------
    g : DistGraph
        The distributed graph..
    nodes : tensor or dict
        Node IDs to sample neighbors from. If it's a dict, it should contain only
        one key-value pair to make this API consistent with dgl.sampling.sample_neighbors.
    fanout : int or dict[etype, int]
        The number of edges to be sampled for each node per edge type.  If an integer
        is given, DGL assumes that the same fanout is applied to every edge type.

        If -1 is given, all of the neighbors will be selected.
    edge_dir : str, optional
        Determines whether to sample inbound or outbound edges.

        Can take either ``in`` for inbound edges or ``out`` for outbound edges.
    prob : str, optional
        Feature name used as the (unnormalized) probabilities associated with each
        neighboring edge of a node.  The feature must have only one element for each
        edge.

        The features must be non-negative floats, and the sum of the features of
        inbound/outbound edges for every node must be positive (though they don't have
        to sum up to one).  Otherwise, the result will be undefined.
    replace : bool, optional
        If True, sample with replacement.

        When sampling with replacement, the sampled subgraph could have parallel edges.

        For sampling without replacement, if fanout > the number of neighbors, all the
        neighbors are sampled. If fanout == -1, all neighbors are collected.
    etype_sorted : bool, optional
        Indicates whether etypes are sorted.

    Returns
    -------
    DGLGraph
        A sampled subgraph containing only the sampled neighboring edges.  It is on CPU.
    """
    if isinstance(fanout, int):
        fanout = F.full_1d(len(g.canonical_etypes), fanout, F.int64, F.cpu())
    else:
        etype_ids = {etype: i for i, etype in enumerate(g.canonical_etypes)}
        fanout_array = [None] * len(g.canonical_etypes)
        for etype, v in fanout.items():
            c_etype = g.to_canonical_etype(etype)
            fanout_array[etype_ids[c_etype]] = v
        assert all(v is not None for v in fanout_array), (
            "Not all etypes have valid fanout. Please make sure passed-in "
            "fanout in dict includes all the etypes in graph. Passed-in "
            f"fanout: {fanout}, graph etypes: {g.canonical_etypes}."
        )
        fanout = F.tensor(fanout_array, dtype=F.int64)

    gpb = g.get_partition_book()
    if isinstance(nodes, dict):
        homo_nids = []
        for ntype in nodes.keys():
            assert (
                ntype in g.ntypes
            ), "The sampled node type {} does not exist in the input graph".format(
                ntype
            )
            if F.is_tensor(nodes[ntype]):
                typed_nodes = nodes[ntype]
            else:
                typed_nodes = toindex(nodes[ntype]).tousertensor()
            homo_nids.append(gpb.map_to_homo_nid(typed_nodes, ntype))
        nodes = F.cat(homo_nids, 0)

    def issue_remote_req(node_ids):
        if prob is not None:
            # See NOTE 1
            _prob = [
                # NOTE (BarclayII)
                # Currently DistGraph.edges[] does not accept canonical etype.
                g.edges[etype].data[prob].kvstore_key
                if prob in g.edges[etype].data
                else ""
                for etype in g.canonical_etypes
            ]
        else:
            _prob = None
        return SamplingRequestEtype(
            node_ids,
            fanout,
            edge_dir=edge_dir,
            prob=_prob,
            replace=replace,
            etype_sorted=etype_sorted,
        )

    def local_access(local_g, partition_book, local_nids):
        etype_offset = gpb.local_etype_offset
        # See NOTE 1
        if prob is None:
            _prob = None
        else:
            _prob = [
                g.edges[etype].data[prob].local_partition
                if prob in g.edges[etype].data
                else None
                for etype in g.canonical_etypes
            ]
        return _sample_etype_neighbors(
            local_g,
            partition_book,
            local_nids,
            etype_offset,
            fanout,
            edge_dir,
            _prob,
            replace,
            etype_sorted=etype_sorted,
        )

    frontier = _distributed_access(g, nodes, issue_remote_req, local_access)
    if not gpb.is_homogeneous:
        return _frontier_to_heterogeneous_graph(g, frontier, gpb)
    else:
        return frontier


def sample_neighbors(g, nodes, fanout, edge_dir="in", prob=None, replace=False):
    """Sample from the neighbors of the given nodes from a distributed graph.

    For each node, a number of inbound (or outbound when ``edge_dir == 'out'``) edges
    will be randomly chosen.  The returned graph will contain all the nodes in the
    original graph, but only the sampled edges.

    Node/edge features are not preserved. The original IDs of
    the sampled edges are stored as the `dgl.EID` feature in the returned graph.

    For heterogeneous graphs, ``nodes`` is a dictionary whose key is node type
    and the value is type-specific node IDs.

    Parameters
    ----------
    g : DistGraph
        The distributed graph..
    nodes : tensor or dict
        Node IDs to sample neighbors from. If it's a dict, it should contain only
        one key-value pair to make this API consistent with dgl.sampling.sample_neighbors.
    fanout : int
        The number of edges to be sampled for each node.

        If -1 is given, all of the neighbors will be selected.
    edge_dir : str, optional
        Determines whether to sample inbound or outbound edges.

        Can take either ``in`` for inbound edges or ``out`` for outbound edges.
    prob : str, optional
        Feature name used as the (unnormalized) probabilities associated with each
        neighboring edge of a node.  The feature must have only one element for each
        edge.

        The features must be non-negative floats, and the sum of the features of
        inbound/outbound edges for every node must be positive (though they don't have
        to sum up to one).  Otherwise, the result will be undefined.
    replace : bool, optional
        If True, sample with replacement.

        When sampling with replacement, the sampled subgraph could have parallel edges.

        For sampling without replacement, if fanout > the number of neighbors, all the
        neighbors are sampled. If fanout == -1, all neighbors are collected.

    Returns
    -------
    DGLGraph
        A sampled subgraph containing only the sampled neighboring edges.  It is on CPU.
    """
    gpb = g.get_partition_book()
    if not gpb.is_homogeneous:
        assert isinstance(nodes, dict)
        homo_nids = []
        for ntype in nodes:
            assert (
                ntype in g.ntypes
            ), "The sampled node type does not exist in the input graph"
            if F.is_tensor(nodes[ntype]):
                typed_nodes = nodes[ntype]
            else:
                typed_nodes = toindex(nodes[ntype]).tousertensor()
            homo_nids.append(gpb.map_to_homo_nid(typed_nodes, ntype))
        nodes = F.cat(homo_nids, 0)
    elif isinstance(nodes, dict):
        assert len(nodes) == 1
        nodes = list(nodes.values())[0]

    def issue_remote_req(node_ids):
        if prob is not None:
            # See NOTE 1
            _prob = g.edata[prob].kvstore_key
        else:
            _prob = None
        return SamplingRequest(
            node_ids, fanout, edge_dir=edge_dir, prob=_prob, replace=replace
        )

    def local_access(local_g, partition_book, local_nids):
        # See NOTE 1
        _prob = (
            [g.edata[prob].local_partition] if prob is not None else None
        )
        return _sample_neighbors(
            local_g,
            partition_book,
            local_nids,
            fanout,
            edge_dir,
            _prob,
            replace,
        )

    frontier = _distributed_access(g, nodes, issue_remote_req, local_access)
    if not gpb.is_homogeneous:
        return _frontier_to_heterogeneous_graph(g, frontier, gpb)
    else:
        return frontier

def _check_location(g, nodes):
    remote_list = []
    partition_book = g.get_partition_book()
    nodes_tensor = toindex(nodes).tousertensor()
    # global node id to partition id
    # nodes가 위치한 partition의 id 모음을 반환
    partition_ids_of_nodes_tensor = partition_book.nid2partid(nodes)
    local_nids = None
    data1 = "0"
    data2 = "0"
    #check partition ids of nodes
    for pid in range(partition_book.num_partitions()):
      # node_ids_each_machine_tensor = nodes_tensor[partition_ids_of_nodes_tensor==pid]
      # node 중 각 머신의 pid에 해당하는 부분만 가지고옴
      node_ids_each_machine_tensor = F.boolean_mask(nodes_tensor, partition_ids_of_nodes_tensor==pid)
      if pid == partition_book.partid and g.local_partition is not None:
          assert local_nids is None
          local_nids = node_ids_each_machine_tensor
          data1 = str(len(local_nids))

      elif len(node_ids_each_machine_tensor) != 0:
          # req = issue_remote_req(node_ids_each_machine_tensor)
          remote_list.append((pid, node_ids_each_machine_tensor))
          data2 = str(len(node_ids_each_machine_tensor))


    f = open("/home/van/disney_workplace/dgl/examples/pytorch/graphsage/dist/dfs_{}_loc_rem.txt".format("1000"),'a+')
    data = str(len(nodes))
    f.write(data+"\n")
    f.write(data1+"\n")
    f.write(data2+"\n")
    f.close()

    return local_nids, remote_list

def _check_location_local(g, partition_book, nodes):
    remote_list = []
    nodes_tensor = toindex(nodes).tousertensor()
    # global node id to partition id
    # nodes가 위치한 partition의 id 모음을 반환
    partition_ids_of_nodes_tensor = partition_book.nid2partid(nodes)

    #check partition ids of nodes
    local_nids = nodes_tensor[partition_ids_of_nodes_tensor==partition_book.partid]
    remote_nids_tensor = nodes_tensor[partition_ids_of_nodes_tensor!=partition_book.partid]
    # local_nids = F.boolean_mask(nodes_tensor, partition_ids_of_nodes_tensor==partition_book.partid)
    # remote_nids_tensor = F.boolean_mask(nodes_tensor, partition_ids_of_nodes_tensor!=partition_book.partid)
    if len(remote_nids_tensor)!=0:
      remote_list = remote_nids_tensor.tolist()

    return local_nids, remote_list

def _check_local(partition_book, nodes):
    # bucket 내의 node들을 check
    remote_list = []
    nodes_tensor = toindex(nodes).tousertensor()
    partition_ids_of_nodes_tensor = partition_book.nid2partid(nodes)
    local_ids = None
    #check partition ids of nodes
    local_ids = F.boolean_mask(nodes_tensor, partition_ids_of_nodes_tensor==partition_book.partid)

    return local_ids

def df_sample_neighbors(g, nodes, fanouts, edge_dir="in", prob=None, replace=False):

    # nodes는 list이면 됨
    gpb = g.get_partition_book()
    if not gpb.is_homogeneous:
        print("Only homogeneous graphs are supported by dfs")
        pass
    elif isinstance(nodes, dict):
        assert len(nodes) == 1
        nodes = list(nodes.values())[0]

    def issue_remote_req(node_ids_each_machine_tensor, bucket, layer_idx):
        # node_ids tensor; 글로벌 아이디
        if prob is not None:
            # See NOTE 1
            _prob = g.edata[prob].kvstore_key
        else:
            _prob = None
        return DFSamplingRequest(
            node_ids_each_machine_tensor, fanouts, edge_dir=edge_dir, prob=_prob, replace=replace, bucket=bucket, layer_idx=layer_idx
        )

    def local_access(local_g, partition_book, local_nids, bucket, layer_idx):
        # See NOTE 1
        _prob = [g.edata[prob].local_partition] if prob is not None else None

        return _df_sample_neighbors(
            local_g,
            partition_book,
            local_nids,
            fanouts,
            edge_dir,
            _prob,
            replace,
            bucket,
            layer_idx
        )

    frontiers = _dfs_distributed_access(g, nodes, issue_remote_req, local_access, fanouts)
    return frontiers

def _distributed_edge_access(g, edges, issue_remote_req, local_access):
    """A routine that fetches local edges from distributed graph.

    The source and destination nodes of local edges are stored in the local
    machine and others are stored on remote machines. This code will issue
    remote access requests first before fetching data from the local machine.
    In the end, we combine the data from the local machine and remote machines.

    Parameters
    ----------
    g : DistGraph
        The distributed graph
    edges : tensor
        The edges to find their source and destination nodes.
    issue_remote_req : callable
        The function that issues requests to access remote data.
    local_access : callable
        The function that reads data on the local machine.

    Returns
    -------
    tensor
        The source node ID array.
    tensor
        The destination node ID array.
    """
    req_list = []
    partition_book = g.get_partition_book()
    edges = toindex(edges).tousertensor()
    partition_id = partition_book.eid2partid(edges)
    local_eids = None
    reorder_idx = []
    for pid in range(partition_book.num_partitions()):
        mask = partition_id == pid
        edge_id = F.boolean_mask(edges, mask)
        reorder_idx.append(F.nonzero_1d(mask))
        if pid == partition_book.partid and g.local_partition is not None:
            assert local_eids is None
            local_eids = edge_id
        elif len(edge_id) != 0:
            req = issue_remote_req(edge_id, pid)
            req_list.append((pid, req))

    # send requests to the remote machine.
    msgseq2pos = None
    if len(req_list) > 0:
        msgseq2pos = send_requests_to_machine(req_list)

    # handle edges in local partition.
    src_ids = F.zeros_like(edges)
    dst_ids = F.zeros_like(edges)
    if local_eids is not None:
        src, dst = local_access(g.local_partition, partition_book, local_eids)
        src_ids = F.scatter_row(
            src_ids, reorder_idx[partition_book.partid], src
        )
        dst_ids = F.scatter_row(
            dst_ids, reorder_idx[partition_book.partid], dst
        )

    # receive responses from remote machines.
    if msgseq2pos is not None:
        results = recv_responses(msgseq2pos)
        for result in results:
            src = result.global_src
            dst = result.global_dst
            src_ids = F.scatter_row(src_ids, reorder_idx[result.order_id], src)
            dst_ids = F.scatter_row(dst_ids, reorder_idx[result.order_id], dst)
    return src_ids, dst_ids


def find_edges(g, edge_ids):
    """Given an edge ID array, return the source and destination
    node ID array ``s`` and ``d`` from a distributed graph.
    ``s[i]`` and ``d[i]`` are source and destination node ID for
    edge ``eid[i]``.

    Parameters
    ----------
    g : DistGraph
        The distributed graph.
    edges : tensor
        The edge ID array.

    Returns
    -------
    tensor
        The source node ID array.
    tensor
        The destination node ID array.
    """

    def issue_remote_req(edge_ids, order_id):
        return EdgesRequest(edge_ids, order_id)

    def local_access(local_g, partition_book, edge_ids):
        return _find_edges(local_g, partition_book, edge_ids)

    return _distributed_edge_access(g, edge_ids, issue_remote_req, local_access)


def in_subgraph(g, nodes):
    """Return the subgraph induced on the inbound edges of the given nodes.

    The subgraph keeps the same type schema and all the nodes are preserved regardless
    of whether they have an edge or not.

    Node/edge features are not preserved. The original IDs of
    the extracted edges are stored as the `dgl.EID` feature in the returned graph.

    For now, we only support the input graph with one node type and one edge type.


    Parameters
    ----------
    g : DistGraph
        The distributed graph structure.
    nodes : tensor or dict
        Node ids to sample neighbors from.

    Returns
    -------
    DGLGraph
        The subgraph.

        One can retrieve the mapping from subgraph edge ID to parent
        edge ID via ``dgl.EID`` edge features of the subgraph.
    """
    if isinstance(nodes, dict):
        assert (
            len(nodes) == 1
        ), "The distributed in_subgraph only supports one node type for now."
        nodes = list(nodes.values())[0]

    def issue_remote_req(node_ids):
        return InSubgraphRequest(node_ids)

    def local_access(local_g, partition_book, local_nids):
        return _in_subgraph(local_g, partition_book, local_nids)

    return _distributed_access(g, nodes, issue_remote_req, local_access)


def _distributed_get_node_property(g, n, issue_remote_req, local_access):
    req_list = []
    partition_book = g.get_partition_book()
    n = toindex(n).tousertensor()
    partition_id = partition_book.nid2partid(n)
    local_nids = None
    reorder_idx = []
    for pid in range(partition_book.num_partitions()):
        mask = partition_id == pid
        nid = F.boolean_mask(n, mask)
        reorder_idx.append(F.nonzero_1d(mask))
        if pid == partition_book.partid and g.local_partition is not None:
            assert local_nids is None
            local_nids = nid
        elif len(nid) != 0:
            req = issue_remote_req(nid, pid)
            req_list.append((pid, req))

    # send requests to the remote machine.
    msgseq2pos = None
    if len(req_list) > 0:
        msgseq2pos = send_requests_to_machine(req_list)

    # handle edges in local partition.
    vals = None
    if local_nids is not None:
        local_vals = local_access(g.local_partition, partition_book, local_nids)
        shape = list(F.shape(local_vals))
        shape[0] = len(n)
        vals = F.zeros(shape, F.dtype(local_vals), F.cpu())
        vals = F.scatter_row(
            vals, reorder_idx[partition_book.partid], local_vals
        )

    # receive responses from remote machines.
    if msgseq2pos is not None:
        results = recv_responses(msgseq2pos)
        if len(results) > 0 and vals is None:
            shape = list(F.shape(results[0].val))
            shape[0] = len(n)
            vals = F.zeros(shape, F.dtype(results[0].val), F.cpu())
        for result in results:
            val = result.val
            vals = F.scatter_row(vals, reorder_idx[result.order_id], val)
    return vals


def in_degrees(g, v):
    """Get in-degrees"""

    def issue_remote_req(v, order_id):
        return InDegreeRequest(v, order_id)

    def local_access(local_g, partition_book, v):
        return _in_degrees(local_g, partition_book, v)

    return _distributed_get_node_property(g, v, issue_remote_req, local_access)


def out_degrees(g, u):
    """Get out-degrees"""

    def issue_remote_req(u, order_id):
        return OutDegreeRequest(u, order_id)

    def local_access(local_g, partition_book, u):
        return _out_degrees(local_g, partition_book, u)

    return _distributed_get_node_property(g, u, issue_remote_req, local_access)


register_service(SAMPLING_SERVICE_ID, SamplingRequest, SubgraphResponse)
register_service(EDGES_SERVICE_ID, EdgesRequest, FindEdgeResponse)
register_service(INSUBGRAPH_SERVICE_ID, InSubgraphRequest, SubgraphResponse)
register_service(OUTDEGREE_SERVICE_ID, OutDegreeRequest, OutDegreeResponse)
register_service(INDEGREE_SERVICE_ID, InDegreeRequest, InDegreeResponse)
register_service(
    ETYPE_SAMPLING_SERVICE_ID, SamplingRequestEtype, SubgraphResponse
)
register_service(DFSAMPLING_SERVICE_ID, DFSamplingRequest, DFSubgraphResponse)